# 1고지

In [1]:
#step 1, 2
%reset
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self, in_data):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2


x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [2]:
#step 3
%reset

import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2


class Exp(Function):
    def forward(self, x):
        return np.exp(x)


A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


In [3]:
#step 4
%reset

import numpy as np

# 1
class Variable:
    def __init__(self, data):
        self.data = data

# 2
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2

# 3
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

# 4-1
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)


f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)


def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))


x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

4.000000000004
3.2974426293330694


In [6]:
#step 6
%reset

import numpy as np

# 1
class Variable:
    def __init__(self, data):
        self.data = data
        # 6.1
        self.grad = None

# 2
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        # 6.2.1 입력 변수를 기억(보관)
        self.input = input
        return output

    def forward(self, x):
        raise NotImplementedError()
    
    # 6.2.2 역전파
    def backward(self, gy):
        raise NotImplementedError()

# 2
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    # 6.3
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

# 3
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    # 6.4
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


# 4
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

# 6
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [7]:
#step 7
%reset

import numpy as np

# 1
class Variable:
    def __init__(self, data):
        self.data = data
        # 6
        self.grad = None
        # 7.1
        self.creator = None
    # 7.1
    def set_creator(self, func):
        self.creator = func
    # 7.3
    def backward(self):
        f = self.creator  # 1. Get a function
        if f is not None:
            x = f.input  # 2. Get the function's input
            x.grad = f.backward(self.grad)  # 3. Call the function's backward
            x.backward()

# 2
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        # 7.1 
        output.set_creator(self) # Set parent(function)
        # 6 
        self.input = input
        # 7.1 
        self.output = output # Set output
        return output

    def forward(self, x):
        raise NotImplementedError()
    
    # 6
    def backward(self, gy):
        raise NotImplementedError()

# 2
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    # 6
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

# 3
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    # 6
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


# 4
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

# 6
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 7.3 backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [8]:
#step 8
%reset

import numpy as np

# 1
class Variable:
    def __init__(self, data):
        self.data = data
        # 6
        self.grad = None
        # 7
        self.creator = None
    # 7
    def set_creator(self, func):
        self.creator = func
    # 7 -> 8.2 (재귀 -> 반복문)
    def backward(self):
        funcs = [self.creator] 
        while funcs:           
            f = funcs.pop()    # 함수를 가져옴
            x, y = f.input, f.output # 함수의 입력과 출력을 가져옴  
            x.grad = f.backward(y.grad) # 자신보다 하나 앞에 놓인 변수의 backward 메서드 호출, 매개 변수로 출력(순전파의 결과)의 미분값을 넣어 줌
                                       
            if x.creator is not None:
                funcs.append(x.creator)  # creator가 계속 안만들어질 때까지 (순전파가 끝날 때까지) funcs 에 creator를 추가 함

# 2
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        # 7 
        output.set_creator(self) # Set parent(function)
        # 6 
        self.input = input
        # 7 
        self.output = output # Set output
        return output

    def forward(self, x):
        raise NotImplementedError()
    
    # 6
    def backward(self, gy):
        raise NotImplementedError()

# 2
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    # 6
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

# 3
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    # 6
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


# 6
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)


y.grad = np.array(1.0)
# 7, 8.3 역전파 (backward 메소드 내부가 바뀌었음)
y.backward()
print(x.grad)

3.297442541400256


In [5]:
#step 9
%reset

import numpy as np

# 1
class Variable:
    def __init__(self, data):
        self.data = data
        # 6
        self.grad = None
        # 7
        self.creator = None
    # 7
    def set_creator(self, func):
        self.creator = func
    # 7 -> 8
    def backward(self):
        funcs = [self.creator] 
        while funcs:           
            f = funcs.pop()    
            x, y = f.input, f.output 
            x.grad = f.backward(y.grad) 
                                       
            if x.creator is not None:
                funcs.append(x.creator)

# 2
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        # 7 
        output.set_creator(self) # Set parent(function)
        # 6 
        self.input = input
        # 7 
        self.output = output # Set output
        return output

    def forward(self, x):
        raise NotImplementedError()
    
    # 6
    def backward(self, gy):
        raise NotImplementedError()

# 2
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    # 6
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

# 3
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    # 6
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def square(x):
    return Square()(x) # 한 줄 작성
def exp(x):
    return Exp()(x)

x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)


3.297442541400256


In [8]:
#step 9
%reset

import numpy as np

# 1
class Variable:
    def __init__(self, data):
        # 9.3 데이터 타입 거르기
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(data)))
        self.data = data
        # 6
        self.grad = None
        # 7
        self.creator = None
    # 7
    def set_creator(self, func):
        self.creator = func
    # 7 -> 8
    def backward(self):
        # 9.2
        if self.grad is None:                   # self.grad 값이 None 이면
            self.grad = np.ones_like(self.data) # 1로 만듦 (아직 까진 self.data가 스칼라값이 기 때문에 ndarray 1 이 됨.)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


x = Variable(np.array(1.0)) # OK
x = Variable(None) # OK

x = Variable(1.0) # ERROR!

TypeError: <class 'float'>은(는) 지원하지 않습니다

In [9]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim) 
print(type(y))

x = np.array(1.0) # 0차원 ndarray
y = x ** 2
print(type(x), x.ndim) 
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [6]:
import numpy as np

# 1
class Variable:
    def __init__(self, data):
        # 9.3 데이터 타입 거르기
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(data)))
        self.data = data
        # 6
        self.grad = None
        # 7
        self.creator = None
    # 7
    def set_creator(self, func):
        self.creator = func
    # 7 -> 8
    def backward(self):
        # 9.2
        if self.grad is None:                   # self.grad 값이 None 이면
            self.grad = np.ones_like(self.data) # 1로 만듦 (아직 까진 self.data가 스칼라값이 기 때문에 ndarray 1 이 됨.)
        
        funcs = [self.creator] 
        while funcs:           
            f = funcs.pop()    
            x, y = f.input, f.output 
            x.grad = f.backward(y.grad) 
                                    
            if x.creator is not None:
                funcs.append(x.creator)

# 2
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        # 9.3
        output = Variable(as_array(y)) # 0차원 ndarray 인스턴스 계산 처리
        # 7 
        output.set_creator(self) # Set parent(function)
        # 6 
        self.input = input
        # 7
        self.output = output # Set output
        return output

    def forward(self, x):
        raise NotImplementedError()
    
    # 6
    def backward(self, gy):
        raise NotImplementedError()

# 2
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    # 6
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

# 3
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    # 6
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

# 9.1
def square(x):
    return Square()(x) # 한 줄 작성
def exp(x):
    return Exp()(x)

# 9.3
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

# 9.1
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

    
x = Variable(np.array(1.0))  # OK
x = Variable(None)  # OK
x = Variable(1.0)  # NG

3.297442541400256


TypeError: <class 'float'>은(는) 지원하지 않습니다

In [ ]:
#step 10
%reset

## 2고지

In [ ]:
#step 11
%reset